Using Huggung face datasets

1. Loading dataset

In [1]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("open-r1/codeforces", split="train")
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00011.parquet:   0%|          | 0.00/290M [00:00<?, ?B/s]

data/train-00001-of-00011.parquet:   0%|          | 0.00/236M [00:00<?, ?B/s]

data/train-00002-of-00011.parquet:   0%|          | 0.00/152M [00:00<?, ?B/s]

...

data/train-00010-of-00011.parquet:   0%|          | 0.00/179M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9556 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/468 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'aliases', 'contest_id', 'contest_name', 'contest_type', 'contest_start', 'contest_start_year', 'index', 'time_limit', 'memory_limit', 'title', 'description', 'input_format', 'output_format', 'interaction_format', 'note', 'examples', 'editorial', 'rating', 'tags', 'testset_size', 'official_tests', 'official_tests_complete', 'input_mode', 'generated_checker', 'executable', 'generated_tests'],
    num_rows: 9556
})


In [20]:
import re
import os
import numpy as np
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    mean_absolute_error,
    mean_squared_error
)

from scipy.sparse import hstack

In [2]:
print(dataset[0])

{'id': '852/A', 'aliases': None, 'contest_id': '852', 'contest_name': 'Bubble Cup X - Finals [Online Mirror]', 'contest_type': 'ICPC', 'contest_start': 1504432800, 'contest_start_year': 2017, 'index': 'A', 'time_limit': 1.0, 'memory_limit': 256.0, 'title': 'Digits', 'description': "John gave Jack a very hard problem. He wrote a very big positive integer A0 on a piece of paper. The number is less than 10200000 . In each step, Jack is allowed to put ' + ' signs in between some of the digits (maybe none) of the current number and calculate the sum of the expression. He can perform the same procedure on that sum and so on. The resulting sums can be labeled respectively by A1, A2 etc. His task is to get to a single digit number.\n\nThe problem is that there is not much blank space on the paper. There are only three lines of space, so he can't perform more than three steps. Since he wants to fill up the paper completely, he will perform exactly three steps.\n\nJack must not add leading zeros

2. Creating a dataframe from the dataset

In [21]:
df = pd.DataFrame(dataset)
df.head()

# Selecting only the needed columns
df = df[['title', 'description', 'input_format', 'output_format', 'rating']]

df.head()


,title,description,input_format,output_format,rating
0,Digits,John gave Jack a very hard problem. He wrote a...,First line contains a positive integer N (1 ≤ ...,"Output exactly three lines, the steps Jack nee...",2500.0
1,Neural Network country,Due to the recent popularity of the Deep learn...,The first line of input contains N (1 ≤ N ≤ 10...,"Output a single integer, the number of paths D...",2000.0
2,Property,Bill is a famous mathematician in BubbleLand. ...,The first line contains one integer number n (...,Output contains n distinct integers separated ...,2100.0
3,Exploration plan,The competitors of Bubble Cup X gathered after...,"The first line contains four integers: V, E, N...",Output a single integer that represents the mi...,2100.0
4,Casinos and travel,John has just bought a new car and is planning...,"In the first line, a positive integer N (1 ≤ N...","Output one number, the answer to the problem m...",2100.0


3. Cleaning and changing according to need

In [22]:
# Function to map numeric rating to Easy/Medium/Hard
def rating_to_class(rating):
    if pd.isna(rating):
        return np.nan
    elif rating <= 1200:
        return "Easy"
    elif rating <= 1800:
        return "Medium"
    else:
        return "Hard"

df['problem_class'] = df['rating'].apply(rating_to_class)
df['problem_score'] = df['rating']

# Drop the original rating column
df = df.drop(columns=['rating'])

df.head()


,title,description,input_format,output_format,problem_class,problem_score
0,Digits,John gave Jack a very hard problem. He wrote a...,First line contains a positive integer N (1 ≤ ...,"Output exactly three lines, the steps Jack nee...",Hard,2500.0
1,Neural Network country,Due to the recent popularity of the Deep learn...,The first line of input contains N (1 ≤ N ≤ 10...,"Output a single integer, the number of paths D...",Hard,2000.0
2,Property,Bill is a famous mathematician in BubbleLand. ...,The first line contains one integer number n (...,Output contains n distinct integers separated ...,Hard,2100.0
3,Exploration plan,The competitors of Bubble Cup X gathered after...,"The first line contains four integers: V, E, N...",Output a single integer that represents the mi...,Hard,2100.0
4,Casinos and travel,John has just bought a new car and is planning...,"In the first line, a positive integer N (1 ≤ N...","Output one number, the answer to the problem m...",Hard,2100.0


In [23]:
df = df.rename(columns={
    'input_format': 'input_description',
    'output_format': 'output_description'
})

df.head()


,title,description,input_description,output_description,problem_class,problem_score
0,Digits,John gave Jack a very hard problem. He wrote a...,First line contains a positive integer N (1 ≤ ...,"Output exactly three lines, the steps Jack nee...",Hard,2500.0
1,Neural Network country,Due to the recent popularity of the Deep learn...,The first line of input contains N (1 ≤ N ≤ 10...,"Output a single integer, the number of paths D...",Hard,2000.0
2,Property,Bill is a famous mathematician in BubbleLand. ...,The first line contains one integer number n (...,Output contains n distinct integers separated ...,Hard,2100.0
3,Exploration plan,The competitors of Bubble Cup X gathered after...,"The first line contains four integers: V, E, N...",Output a single integer that represents the mi...,Hard,2100.0
4,Casinos and travel,John has just bought a new car and is planning...,"In the first line, a positive integer N (1 ≤ N...","Output one number, the answer to the problem m...",Hard,2100.0


In [24]:
# Drop rows where the description is empty or just whitespace
df = df[df['description'].str.strip() != '']

# Optional: reset index
df = df.reset_index(drop=True)

# Drop rows with missing labels
df = df.dropna(subset=['problem_class', 'problem_score']).reset_index(drop=True)


print("Cleaned dataset shape:", df.shape)


Cleaned dataset shape: (9309, 6)


In [25]:
print(df['problem_class'].isna().sum())
print(df['problem_score'].isna().sum())


0
0


In [27]:
print("NaN in X:", np.isnan(X.data).sum())
print("NaN in y_class:", y_class.isna().sum())
print("NaN in y_score:", y_score.isna().sum())


NaN in X: 0
NaN in y_class: 0
NaN in y_score: 0


In [40]:
# Columns: 'title', 'description', 'input_description', 'output_description', 'problem_class', 'problem_score'

# Combine text fields
text_columns = ['title', 'description', 'input_description', 'output_description']
df['combined_text'] = df[text_columns].fillna('').agg(' '.join, axis=1)

# ====== Step 2: Clean the combined text ======
def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces/newlines
    text = re.sub(r"[^a-z0-9+\-*/=^<>% ]", "", text)  # keep letters, numbers, math symbols, spaces
    return text

df['combined_text'] = df['combined_text'].apply(clean_text)

# ====== Step 3: Create additional features ======
# 1. Text length and word count
df['text_length'] = df['combined_text'].apply(len)
df['word_count'] = df['combined_text'].apply(lambda x: len(x.split()))

# 2. Count mathematical symbols
math_symbols = '+-*/=^<>%'
df['math_symbols'] = df['combined_text'].apply(lambda x: sum(x.count(s) for s in math_symbols))

# 3. Keyword frequency
keywords = [
    # Core paradigms
    'dp', 'dynamic programming', 'recursion', 'backtracking',
    'greedy', 'divide and conquer',

    # Graphs & trees
    'graph', 'tree', 'dfs', 'bfs',
    'shortest path', 'dijkstra', 'bellman ford',
    'topological', 'mst', 'lca',

    # Data structures
    'segment tree', 'fenwick', 'binary indexed tree',
    'heap', 'priority queue', 'deque',

    # Searching & optimization
    'binary search', 'two pointers', 'sliding window',

    # Advanced / hard signals
    'bitmask', 'bit manipulation',
    'flow', 'max flow', 'min cost',
    'matching',
    'combinatorics', 'probability',
    'modulo'
]

for kw in keywords:
    col_name = f"kw_{kw.replace(' ', '_')}"
    df[col_name] = df['combined_text'].apply(lambda x: x.count(kw))


# ====== Step 4: TF-IDF vectors ======
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)
)

X_tfidf = tfidf.fit_transform(df['combined_text'])

# ====== Step 5: Combine TF-IDF with extra features ======
from sklearn.preprocessing import StandardScaler

extra_feature_cols = (
    ['text_length', 'word_count', 'math_symbols'] +
    [f"kw_{kw.replace(' ', '_')}" for kw in keywords]
)

extra_features = df[extra_feature_cols].values


scaler = StandardScaler()
extra_features_scaled = scaler.fit_transform(extra_features)

X = hstack([X_tfidf, extra_features_scaled])


print("Feature matrix shape:", X.shape)
print("Preview dataframe with extra features:")
df.head()


Feature matrix shape: (9309, 5037)
Preview dataframe with extra features:


,title,description,input_description,output_description,problem_class,problem_score,combined_text,text_length,word_count,math_symbols,...,kw_sliding_window,kw_bitmask,kw_bit_manipulation,kw_flow,kw_max_flow,kw_min_cost,kw_matching,kw_combinatorics,kw_probability,kw_modulo
0,Digits,John gave Jack a very hard problem. He wrote a...,First line contains a positive integer N (1 ≤ ...,"Output exactly three lines, the steps Jack nee...",Hard,2500.0,digits john gave jack a very hard problem he w...,1541,292,7,...,0,0,0,0,0,0,0,0,0,0
1,Neural Network country,Due to the recent popularity of the Deep learn...,The first line of input contains N (1 ≤ N ≤ 10...,"Output a single integer, the number of paths D...",Hard,2000.0,neural network country due to the recent popul...,1496,292,1,...,0,0,0,0,0,0,0,0,0,1
2,Property,Bill is a famous mathematician in BubbleLand. ...,The first line contains one integer number n (...,Output contains n distinct integers separated ...,Hard,2100.0,property bill is a famous mathematician in bub...,1737,360,50,...,0,0,0,0,0,0,0,0,0,0
3,Exploration plan,The competitors of Bubble Cup X gathered after...,"The first line contains four integers: V, E, N...",Output a single integer that represents the mi...,Hard,2100.0,exploration plan the competitors of bubble cup...,1989,392,3,...,0,0,0,0,0,0,0,0,0,0
4,Casinos and travel,John has just bought a new car and is planning...,"In the first line, a positive integer N (1 ≤ N...","Output one number, the answer to the problem m...",Hard,2100.0,casinos and travel john has just bought a new ...,1601,325,4,...,0,0,0,0,0,0,0,0,0,2


In [41]:
import os
import joblib

os.makedirs("models", exist_ok=True)

joblib.dump(tfidf, "models/tfidf.pkl")
joblib.dump(scaler, "models/scaler.pkl")



['models/scaler.pkl']

In [42]:
from sklearn.model_selection import train_test_split

y_class = df['problem_class']
y_score = df['problem_score']

X_train, X_test, y_class_train, y_class_test = train_test_split(
    X, y_class, test_size=0.2, random_state=42, stratify=y_class
)

_, _, y_score_train, y_score_test = train_test_split(
    X, y_score, test_size=0.2, random_state=42
)


CLASSIFICATION

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

clf = LogisticRegression(max_iter=1000, n_jobs=-1)
clf.fit(X_train, y_class_train)

preds = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_class_test, preds))
print("Confusion Matrix:\n", confusion_matrix(y_class_test, preds))
import joblib
import os

os.makedirs("models", exist_ok=True)
joblib.dump(clf, "models/classifier.pkl")



Accuracy: 0.6031149301825993
Confusion Matrix:
 [[277  86  89]
 [ 54 724 120]
 [ 99 291 122]]


['models/classifier.pkl']

RANDOM FORREST

In [45]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_train_log = np.log1p(y_score_train)
y_test_log = np.log1p(y_score_test)

reg = Ridge(alpha=10.0)
reg.fit(X_train, y_train_log)

preds = np.expm1(reg.predict(X_test))
preds = np.clip(preds, 800, 3500)

print("MAE:", mean_absolute_error(y_score_test, preds))
print("RMSE:", np.sqrt(mean_squared_error(y_score_test, preds)))
joblib.dump(reg, "models/regressor.pkl")




MAE: 605.8943508248238
RMSE: 737.4840280554223


['models/regressor.pkl']